In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
import os

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
sys.path.insert(0, "./")
sys.path.insert(0, "/dadendev/nlp/examples/text_summarization/BertSum/")

In [4]:
print(sys.path)

['/dadendev/nlp/examples/text_summarization/BertSum/', './', '/dadendev/nlp', '/dadendev/anaconda3/envs/cm3/lib/python36.zip', '/dadendev/anaconda3/envs/cm3/lib/python3.6', '/dadendev/anaconda3/envs/cm3/lib/python3.6/lib-dynload', '', '/home/daden/.local/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/pyrouge-0.1.3-py3.6.egg', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions', '/home/daden/.ipython']


In [5]:
#"""
from utils_nlp.dataset.cnndm import CNNDMSummarization

train_dataset, test_dataset = CNNDMSummarization(top_n=5)

len(train_dataset)

len(test_dataset)

test_dataset[0]
#"""

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]


[['marseille',
  ',',
  'france',
  '(',
  'cnn',
  ')',
  'the',
  'french',
  'prosecutor',
  'leading',
  'an',
  'investigation',
  'into',
  'the',
  'crash',
  'of',
  'germanwings',
  'flight',
  '9525',
  'insisted',
  'wednesday',
  'that',
  'he',
  'was',
  'not',
  'aware',
  'of',
  'any',
  'video',
  'footage',
  'from',
  'on',
  'board',
  'the',
  'plane',
  '.'],
 ['marseille',
  'prosecutor',
  'brice',
  'robin',
  'told',
  'cnn',
  'that',
  '``',
  'so',
  'far',
  'no',
  'videos',
  'were',
  'used',
  'in',
  'the',
  'crash',
  'investigation',
  '.',
  '``'],
 ['he',
  'added',
  ',',
  '``',
  'a',
  'person',
  'who',
  'has',
  'such',
  'a',
  'video',
  'needs',
  'to',
  'immediately',
  'give',
  'it',
  'to',
  'the',
  'investigators',
  '.',
  '``'],
 ['robin',
  "'s",
  'comments',
  'follow',
  'claims',
  'by',
  'two',
  'magazines',
  ',',
  'german',
  'daily',
  'bild',
  'and',
  'french',
  'paris',
  'match',
  ',',
  'of',
  'a',
  'cel

In [6]:
from utils_nlp.models.transformers.extractive_summarization import ExtSumProcessor, ExtractiveSummarizer

I1026 02:18:57.081262 139722144081728 file_utils.py:39] PyTorch version 1.2.0 available.
I1026 02:18:57.117647 139722144081728 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1026 02:18:57.134480 139722144081728 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [7]:
processor = ExtSumProcessor()

I1026 02:18:57.327846 139722144081728 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at ./5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [8]:
#"""
ext_sum_train = processor.preprocess(list(train_dataset), list(train_dataset.get_target()))

ext_sum_test = processor.preprocess(list(test_dataset), list(test_dataset.get_target()))

#type(ext_sum_train)
#"""

In [9]:
import glob
BERT_DATA_PATH = BERT_DATA_PATH="./bert_data/"
pts = sorted(glob.glob(BERT_DATA_PATH + 'cnndm.train' + '.[0-9]*.pt'))

In [10]:
pts[0:2]

['./bert_data/cnndm.train.0.bert.pt', './bert_data/cnndm.train.1.bert.pt']

In [11]:
import torch
def get_dataset(file_list):
    #random.shuffle(file_list)
    for file in file_list:
        yield torch.load(file)

In [12]:
from bertsum.models.data_loader  import DataIterator
from bertsum.models import  model_builder, data_loader
class Bunch(object):
    """ Class which convert a dictionary to an object """

    def __init__(self, adict):
        self.__dict__.update(adict)
def get_data_loader(data_files, device, is_labeled=False, batch_size=3000):
    """
    Function to get data iterator over a list of data objects.

    Args:
        dataset (list of objects): a list of data objects.
        is_test (bool): it specifies whether the data objects are labeled data.
        batch_size (int): number of tokens per batch.
        
    Returns:
        DataIterator

    """
    args = Bunch({})
    args.use_interval = True
    args.batch_size = batch_size
    data_iter = None
    data_iter  = data_loader.Dataloader(args, get_dataset(data_files), args.batch_size,device, shuffle=False, is_test=is_labeled)
    return data_iter

In [13]:
device = torch.device("cuda:{}".format(0)) 
def train_iter_func():
    return get_data_loader(pts[0:2], device, is_labeled=True)

In [14]:
summarizer = None
summarizer = ExtractiveSummarizer()

I1026 02:18:58.823258 139722144081728 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at ./a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1026 02:18:58.824696 139722144081728 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1026 02:18:58.951738 139722144081728 modeling_utils.

In [15]:
# notebook parameters
DATA_FOLDER = "./temp"
CACHE_DIR = "./temp"
DEVICE = "cuda"
NUM_EPOCHS = 1
BATCH_SIZE = 64
NUM_GPUS = 1
MAX_LEN = 150

In [16]:
from utils_nlp.common.timer import Timer
with Timer() as t:
    summarizer.fit(
            train_iter_func,
            device= DEVICE,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            num_gpus=NUM_GPUS,
            max_steps=1e2,
            verbose=True,
        )
train_time = t.interval / 3600    
    


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

0.34944987297058105
loss: 62.9429931640625 time (s): 0.34944987297058105
5.891521215438843
loss: 37.97108459472656 time (s): 5.891521215438843
6.03455924987793
loss: 43.47504425048828 time (s): 6.03455924987793


In [17]:
import torch
from utils_nlp.models.bert.extractive_text_summarization import get_data_iter
import os

test_dataset=[]
for i in range(0,6):
    filename = os.path.join(BERT_DATA_PATH, "cnndm.test.{0}.bert.pt".format(i))
    test_dataset.extend(torch.load(filename))

In [19]:
prediction = summarizer.predict(get_data_iter(test_dataset),
                               device=DEVICE,)

[[1.1677501 1.1653192 1.175272  1.1832693 1.1694225 1.1368204 1.1037135
  1.0816852 1.0714864 1.0696733 1.0689341 1.0639383 1.0556616 1.0476966
  1.0439192 1.0460278 1.0511273 0.        0.       ]
 [1.1750555 1.1707659 1.1813774 1.1912082 1.1794333 1.146521  1.1114887
  1.087969  1.0760933 1.0731065 1.0721755 1.0663466 1.0569862 1.0483811
  1.0443294 1.0458696 1.0510468 1.0559808 1.0585443]
 [1.1648579 1.1615411 1.1736461 1.182917  1.1719552 1.1382117 1.1033176
  1.0802764 1.0699608 1.0675933 1.0672765 1.0623417 1.0524166 1.0442766
  1.0405976 1.0426865 1.0478616 1.0530761 0.       ]
 [1.1938438 1.1884855 1.1976215 1.2067728 1.1929277 1.1589307 1.1225741
  1.0984077 1.0867304 1.0846443 1.0841185 1.0789733 1.068714  1.0586185
  0.        0.        0.        0.        0.       ]
 [1.1661083 1.1612492 1.1697978 1.1759909 1.1647654 1.1327995 1.1000191
  1.0791873 1.0684466 1.0672373 1.0679868 1.064056  1.0554876 1.0475358
  1.0438521 0.        0.        0.        0.       ]]
[[1.1875246 1.

[[1.1580768 1.1546454 1.1636755 1.1713219 1.1595958 1.129284  1.0970718
  1.0762812 1.0664221 1.065048  1.0646732 1.0602422 1.0518239 1.0442685
  1.0406064 1.0418304 1.0465701 0.        0.        0.        0.
  0.        0.        0.       ]
 [1.1816815 1.1765453 1.1866174 1.1946247 1.1819737 1.1467488 1.1111163
  1.0875686 1.0764656 1.0739118 1.073026  1.0680461 1.0587199 1.0501839
  1.046495  1.048328  1.0536666 1.0591205 0.        0.        0.
  0.        0.        0.       ]
 [1.1721478 1.1658251 1.1746186 1.18349   1.1727731 1.1411355 1.108191
  1.0857495 1.07448   1.0713452 1.0701457 1.0655565 1.0563318 1.0484395
  1.0447686 1.0464734 1.0517408 0.        0.        0.        0.
  0.        0.        0.       ]
 [1.156463  1.152319  1.160788  1.1685114 1.1565189 1.1263882 1.0955616
  1.0749364 1.0648336 1.0625143 1.061422  1.0574805 1.0499368 1.042669
  1.0394233 1.0406629 1.0450622 1.0495256 0.        0.        0.
  0.        0.        0.       ]
 [1.1698468 1.1660281 1.1758661 1.

[[1.1672728 1.1656395 1.1772993 1.1858649 1.1735258 1.1400894 1.1056526
  1.0826112 1.0717909 1.0683889 1.0659854 1.0600418 1.051377  1.04358
  1.0407546 1.0423039 1.0473907 1.0523226 1.0547266 1.0557818 1.0575589
  1.0614295 0.        0.       ]
 [1.150325  1.148191  1.1590457 1.1680427 1.1566468 1.1255726 1.0954452
  1.0745788 1.0643306 1.060942  1.058763  1.0532348 1.0451624 1.0386227
  1.0360504 1.0378407 1.0426463 1.0472093 1.0496233 1.0506524 1.0520213
  0.        0.        0.       ]
 [1.1482093 1.145635  1.1562245 1.1658132 1.1566485 1.1272873 1.0961789
  1.0751622 1.0647849 1.0613083 1.0593895 1.0541266 1.0459005 1.0389307
  1.03625   1.0379467 1.0425514 1.0466912 1.0494215 1.0503145 1.0514939
  1.0544832 1.0609291 1.0697025]
 [1.1686338 1.1634858 1.1723843 1.1805615 1.1690657 1.1363304 1.1029818
  1.0808774 1.0697534 1.0663385 1.0656462 1.0606651 1.0530171 1.045526
  1.0420983 1.0441772 1.0494392 1.0548873 0.        0.        0.
  0.        0.        0.       ]
 [1.160445  1.

[[1.1780031 1.1733578 1.1840404 1.19387   1.1821567 1.1478434 1.1118851
  1.0876853 1.0773292 1.0747732 1.0748838 1.0695754 1.0599539 1.0513524
  1.047131  1.0487261 1.0544469 0.       ]
 [1.1703753 1.1666003 1.175345  1.1838946 1.1706364 1.138376  1.1047312
  1.0827863 1.0729616 1.0714424 1.0717679 1.0665892 1.0580926 1.050002
  0.        0.        0.        0.       ]
 [1.1684434 1.1641092 1.1739823 1.1815119 1.1711943 1.1378288 1.1052046
  1.0833175 1.072422  1.0701556 1.0694416 1.0649389 1.0561054 1.0480186
  1.0445046 1.0465231 1.0521917 0.       ]
 [1.1874659 1.1842349 1.1971172 1.2079235 1.1949425 1.1578094 1.1190784
  1.0935817 1.0817513 1.0790393 1.0786226 1.0725487 1.062488  1.0530692
  1.0491704 1.0509043 1.0568869 1.0628493]
 [1.1862319 1.1818776 1.1905531 1.1986487 1.1845468 1.150577  1.1147417
  1.0901939 1.0786004 1.0756291 1.0749673 1.0705601 1.0617019 1.053151
  1.0494931 1.0516382 1.057429  0.       ]]
[[1.18455   1.1804587 1.1897806 1.1982723 1.1854905 1.151333  1.11

[[1.1698228 1.1661043 1.1762784 1.1853486 1.1735361 1.1399157 1.1061329
  1.0834488 1.0736991 1.0719409 1.0715246 1.0668477 1.0575045 1.0488973
  1.0452029 1.0472383 0.        0.       ]
 [1.1960676 1.1916723 1.2021546 1.2128708 1.199506  1.1623422 1.1232877
  1.0976712 1.086281  1.0850179 1.085532  1.0797937 1.0685506 1.0586826
  0.        0.        0.        0.       ]
 [1.1704038 1.1649016 1.1747351 1.1840938 1.1729548 1.1405728 1.1064197
  1.0846469 1.0743003 1.0728241 1.0733166 1.0688934 1.059711  1.0509413
  0.        0.        0.        0.       ]
 [1.1706715 1.1666707 1.178094  1.1877062 1.1756464 1.1426352 1.108415
  1.0851184 1.0743653 1.071885  1.0714883 1.0669415 1.0583736 1.049874
  1.0457926 1.0472734 1.0524312 0.       ]
 [1.1554242 1.1510313 1.1589519 1.1665778 1.1555799 1.1256192 1.0946608
  1.0738779 1.0635399 1.0602963 1.0598992 1.0564215 1.0489922 1.0419872
  1.0389745 1.040294  1.0447087 1.0493959]]
[[1.1754916 1.1728393 1.1849339 1.1939642 1.1827241 1.1481714 1.11

...

[[1.1593453 1.1557529 1.1649296 1.1717771 1.1596612 1.1272981 1.0964227
  1.076549  1.0669681 1.0643586 1.0637953 1.0588073 1.0507059 1.0432369
  1.0404365 1.0423898 1.0471671 1.0517708 0.        0.        0.
  0.       ]
 [1.1709151 1.1683472 1.178189  1.187283  1.17525   1.1420141 1.1084342
  1.085909  1.0742327 1.0703312 1.0676881 1.0615985 1.0521013 1.044547
  1.0412927 1.0429469 1.0478426 1.0526447 1.0553344 1.0567429 1.0585825
  0.       ]
 [1.1566143 1.1554583 1.1663359 1.1760879 1.1651914 1.1331786 1.1003929
  1.0786636 1.0680544 1.0639768 1.0625857 1.0571092 1.0489484 1.042062
  1.0388283 1.0402722 1.044295  1.048815  1.0509148 1.0515928 1.0530413
  1.0564502]
 [1.1685554 1.1638727 1.1739141 1.1812879 1.1698508 1.1370456 1.1032645
  1.0812831 1.0712047 1.0694947 1.0696834 1.0644156 1.0556645 1.0476772
  1.0434715 1.04538   0.        0.        0.        0.        0.
  0.       ]
 [1.1788648 1.1749029 1.1855204 1.1957343 1.183254  1.1485775 1.1124974
  1.0888147 1.0785757 1.0774

[[1.163403  1.1598704 1.1694934 1.1772511 1.1646141 1.1333678 1.100936
  1.079307  1.0700122 1.0678129 1.0672071 1.0621457 1.0528812 1.0452342
  1.0418624 1.0435762 1.0490456 0.       ]
 [1.1703293 1.1668899 1.176639  1.1852894 1.1723145 1.1392653 1.1048566
  1.0823338 1.0716289 1.0692124 1.0686948 1.0634335 1.0544672 1.0464231
  1.0429554 1.044882  1.0499463 1.0549268]
 [1.1759112 1.1703317 1.1807972 1.1901655 1.1787466 1.144428  1.108871
  1.0856919 1.0759193 1.0755328 1.0760374 1.0712796 1.0613176 0.
  0.        0.        0.        0.       ]
 [1.1922623 1.1881464 1.1994616 1.2075706 1.1932752 1.1557184 1.1180807
  1.0942765 1.0844551 1.0836934 1.0847763 1.078629  1.0669336 0.
  0.        0.        0.        0.       ]
 [1.1801887 1.1747028 1.1844354 1.1917825 1.1793634 1.1451752 1.1109616
  1.0888419 1.0788411 1.0777104 1.0778589 1.0721389 1.0620201 1.0528276
  0.        0.        0.        0.       ]]
[[1.1657903 1.1608789 1.1705451 1.1799313 1.1684747 1.136118  1.1024472
  1.0799

[[1.170255  1.1659663 1.1769745 1.1860996 1.1751902 1.1413867 1.1069624
  1.0838164 1.0737109 1.0715982 1.0716908 1.0675863 1.058299  1.0496987
  1.0460944 0.        0.        0.       ]
 [1.1752774 1.1719513 1.1819901 1.1896224 1.1768789 1.1427811 1.108832
  1.0868037 1.0773718 1.0763466 1.0763103 1.0709836 1.0611674 1.052452
  0.        0.        0.        0.       ]
 [1.2130291 1.2079074 1.2179445 1.2280728 1.2135587 1.1760418 1.1347064
  1.1073956 1.0952911 1.0937215 0.        0.        0.        0.
  0.        0.        0.        0.       ]
 [1.1945512 1.1907668 1.2005379 1.2085509 1.1955283 1.1590024 1.1215622
  1.0960356 1.0846939 1.0820097 1.0809064 1.0754015 1.0652753 1.0557778
  1.0516948 1.0541006 1.060639  0.       ]
 [1.1835619 1.179721  1.1906612 1.1995771 1.186794  1.1508604 1.1141328
  1.0896902 1.0780323 1.0754873 1.074065  1.06858   1.0593688 1.0507944
  1.0471208 1.0491204 1.0551434 1.0615337]]
[[1.1992183 1.1916575 1.2007495 1.2086978 1.1969676 1.1606244 1.1226381
 

[[1.16686   1.1627344 1.1728532 1.1811973 1.1702282 1.1379632 1.1044514
  1.0824326 1.0720983 1.0699883 1.0701799 1.0657024 1.0577787 1.0493773
  1.0455085 1.047367  0.        0.        0.       ]
 [1.1574018 1.1533397 1.1622702 1.1685348 1.1573118 1.1272938 1.0964421
  1.0762498 1.0659733 1.0637034 1.0626162 1.0578108 1.0499096 1.0432427
  1.0398666 1.0418628 1.0467248 1.051483  1.054306 ]
 [1.1795785 1.1761023 1.186912  1.1969068 1.1844599 1.1487857 1.1128155
  1.0887232 1.0771029 1.0740697 1.0731341 1.0680435 1.058768  1.0504072
  1.0463998 1.0479867 1.0536331 1.0587804 0.       ]
 [1.1664665 1.1615186 1.1722153 1.1825583 1.1713036 1.1377991 1.1034583
  1.0810404 1.0710382 1.0698572 1.0698137 1.0650911 1.055966  1.0477245
  1.0439353 0.        0.        0.        0.       ]
 [1.1836717 1.1805123 1.1919913 1.1994302 1.187069  1.1502919 1.1130128
  1.0891472 1.0783182 1.0758077 1.0762032 1.0706948 1.0609246 1.0526567
  1.0485038 1.0504786 1.0560647 0.        0.       ]]
[[1.1791857 1.

[[1.1673328 1.1628357 1.1728365 1.1814666 1.1691337 1.1364444 1.1026524
  1.080059  1.0698013 1.0679461 1.0682526 1.0639735 1.05545   1.0472834
  1.0434302 1.0450319 1.0504022 0.        0.       ]
 [1.1588726 1.155991  1.165724  1.1730864 1.1615422 1.1293397 1.0974313
  1.0767077 1.0669677 1.0645189 1.0638295 1.0588771 1.050008  1.0426273
  1.0392169 1.041301  1.0458218 1.0507703 1.0531998]
 [1.198158  1.1904218 1.1985812 1.2062509 1.1935307 1.158339  1.1207788
  1.0955433 0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.       ]
 [1.2036841 1.1982536 1.208348  1.2154396 1.2003766 1.1626592 1.1231865
  1.0976042 1.0856462 1.0850655 1.0863111 0.        0.        0.
  0.        0.        0.        0.        0.       ]
 [1.1634002 1.1591038 1.1683466 1.1774144 1.1662323 1.1342905 1.1025496
  1.0809062 1.070615  1.0693222 1.069299  1.0648515 1.0568047 1.0486447
  1.0449787 0.        0.        0.        0.       ]]
[[1.1617231 1.1575178 1.1661

In [20]:
prediction[0]

[autoreload of utils_nlp.models.transformers.extractive_summarization failed: Traceback (most recent call last):
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions/

'andrew mogni , 20 , from glen ellyn , illinois , a university of iowa student has died nearly three months after a fall in rome in a suspected robbery<q>he was flown back to chicago via air ambulance on march 20 , but he died on sunday .<q>he was taken to a medical facility in the chicago area , close to his family home in glen ellyn .'

In [21]:
test_dataset[0]['tgt_txt']

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'